In [ ]:
import os, shutil
import pandas as pd
import numpy  as np
from tqdm import tqdm
from WCFAdox import PCAX
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.font_manager import FontProperties
from tabulate import tabulate

# Data (Download & Preprocessing)

In [ ]:
# Transform date from YYYYMMDD to YYYY-MM-DD
def date_transform(text):
    date = text[:4] + '-' + text[4:6] + '-' + text[6:]
    return date

# Clear and recreate a folder
def clear_construct_folder(foldername):
    try:
        shutil.rmtree(foldername)  # Remove folder if exists
    except:
        os.makedirs(foldername)    # Create folder if removal fails
    else:
        os.makedirs(foldername)    # Create folder after successful removal

# Retrieve broker-specific transaction data
def get_broker_data(startDate, endDate, stock, broker):
    PX = PCAX("10.216.8.148")
    df = PX.Sil_Data("個股券商進出明細", "D", stock, startDate, endDate, isst="Y")
    df = df[ df["券商名稱"] == broker ]
    df["日期"] = df["日期"].astype(str).apply(date_transform)
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    return df

# Retrieve stock daily and odd-lot trading data
def get_stock_data(startDate, endDate, stock):
    PX = PCAX("10.216.8.148")
    df = PX.Sil_Data("日收盤表排行", "D", stock, startDate, endDate, isst="Y")
    df["日期"] = df["日期"].astype(str).apply(date_transform)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y-%m-%d", yearfirst=True)
    df        = df.sort_values(by="日期")
    df["日期"] = df["日期"].dt.strftime("%Y-%m-%d")

    df1 = PX.Sil_Data("日盤後零股交易行情", "D", stock, startDate, endDate, isst="Y")
    df1["日期"] = df1["日期"].astype(str).apply(date_transform)
    df1["日期"] = pd.to_datetime(df1["日期"], format="%Y-%m-%d", yearfirst=True)
    df1        = df1.sort_values(by="日期")
    df1["日期"] = df1["日期"].dt.strftime("%Y-%m-%d")
    df1 = df1[ ["日期", "成交股數"] ]
    
    df  = pd.merge(df1, df, how="outer", on="日期")
    df  = df.reset_index(drop=True)
    return df

# Equity Value Calculation

In [ ]:
# Simulate broker's equity based on trading behavior
def broker_equity(df):
    df1            = df.copy()
    df1["holding"] = 0
    df1["cash"]    = 0
    df1["equity"]  = 0
    df1            = df1.apply(pd.to_numeric, errors='ignore')

    for index, row in df1.iterrows():
        if (index == 0):
            continue
        df1.loc[index, "holding"] = df1.loc[(index - 1), "holding"] + df1.loc[index, "張增減"]
        df1.loc[index, "cash"]    = df1.loc[(index - 1), "cash"]    - df1.loc[index, "買張"] * df1.loc[index, "買均價"] + df1.loc[index, "賣張"] * df1.loc[index, "賣均價"]
        df1.loc[index, "equity"]  = df1.loc[index, "cash"]          + df1.loc[index, "holding"] * df1.loc[index, "收盤價"]
    return df1

# Simulate strategy by tracking position and cash
def calculation(df1):
    df1["holding"] = 0
    df1["cash"]    = 0
    df1            = df1.apply(pd.to_numeric, errors='ignore')

    for index, row in df1.iterrows():
        if (index == 0):
            continue
        df1.loc[index, "holding"] = df1.loc[(index - 1), "holding"] + df1.loc[index, "volume"]
        df1.loc[index, "cash"]    = df1.loc[(index - 1), "cash"]    - df1.loc[index, "volume"] * df1.loc[index, "開盤價"]
    return df1

# Build strategy summary table
def strategy_sheet(df1):
    s1            = df1[["日期", "開盤價", "最高價", "最低價", "收盤價", "volume", "holding", "cash"]].copy()
    s1["equity"]  = s1["cash"] + s1["holding"] * s1["收盤價"]
    return s1

# Strategy

In [ ]:
# Strategy: mimic previous day's net volume
def strategy(df):
    sName = "模仿前一日買賣超"

    df1           = df.apply(pd.to_numeric, errors='ignore').copy()
    df1["volume"] = df1["張增減"].shift(1, fill_value=0)

    df1        = calculation(df1)
    plot_sheet = strategy_sheet(df1)
    return plot_sheet, sName

# Plot broker vs. strategies equity curve
def plot(folder, result_broker, result_strategy, name, stock, broker):
    plt.figure(figsize=(20, 6))
    font    = FontProperties(fname=r'NotoSansTC-Black.otf')
    fig, ax = plt.subplots()

    ax.set_xlabel ('日期', fontproperties=font, fontsize=10)
    ax.set_ylabel ('千元', fontproperties=font, fontsize=10)

    x      = result_broker["日期"]
    y1, y2 = result_broker["equity"], result_strategy["equity"]

    ax.plot(x, y1, label="外資券商",          color="darkorange", linewidth=1)
    ax.plot(x, y2, label=f"交易策略-{name1}", color="cadetblue",  linewidth=3)

    title = f"{stock}_{broker}"
    plt.title  (title, fontproperties=font, fontsize=20)            # Chart title
    plt.axhline(0, color='black', linestyle='--')
    ax.legend (prop={'fname': 'NotoSansTC-Black.otf', 'size': 10})  # Legend

    axes     = plt.gca()
    interval = x.shape[0] // 4                                      # Set x-axis tick interval
    axes.xaxis.set_major_locator(mdates.DayLocator(interval=interval))

    addr = f"{folder}\\{title}.png"
    plt.savefig(addr)
    plt.clf()
    plt.close("all")

# Example

In [ ]:
startDate = "20220101"
endDate   = "20221227"

brokers = ["美林", "美商高盛", "摩根大通", "法銀巴黎", "新加坡商瑞銀"]
stocks  = ["1229", "2409", "5009", "1304", "2498", "2615"]
folder  = "C:\\Users\\user.Y220026097\\Desktop\\Equity"
clear_construct_folder(folder)

for stock in tqdm(stocks):
    df_stock = get_stock_data(startDate, endDate, stock)
    folder = f"C:\\Users\\user.Y220026097\\Desktop\\Equity\\{stock}"
    clear_construct_folder(folder)

for broker in brokers:
    df_broker = get_broker_data(startDate, endDate, stock, broker)
    df        = pd.merge(df_broker, df_stock, how="outer", on="日期")

    result_broker         = broker_equity(df)
    result_strategy, name = strategy(df)
    plot(folder, result_broker, result_strategy, name, stock, broker)